# Install and import packages 



In [4]:
import os
import pandas as pd
import numpy as np

In [33]:
!pip install sentence_transformers
!pip install scikit-learn
!pip install unzip

In [5]:
from sentence_transformers import SentenceTransformer
from collections import Counter



In [35]:
#Upload csv.zip and text.zip from the drop box folder CogAI/synthea_colab
!unzip csv.zip
!unzip text.zip

Archive:  csv.zip
replace 2022_04_04T07_13_58Z/claims.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  text.zip
replace text/Sheryl275_Gerlach374_7892936b-d8b1-f3a4-7c08-377ac61de459.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Load the model and encode any text

In [6]:
bioBERT = 'emilyalsentzer/Bio_ClinicalBERT'
model = SentenceTransformer(bioBERT)
example_embeddings = model.encode("we can embed anything")

Some weights of the model checkpoint at C:\Users\Jerem/.cache\torch\sentence_transformers\emilyalsentzer_Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# If you run the code to process the .zips this will encode a single patients record
# patient_text_embed = model.encode(texts[0])
patient_procedures_embed = model.encode(patients_info[0]['procedures'])

NameError: name 'patients_info' is not defined

In [ ]:
patients_info[0]['procedures'].shape

(54,)

In [ ]:
patient_procedures_embed.shape

(54, 768)

In [ ]:
patient_text_embed = model.encode(texts[0])
patient_text_embed.shape

(682, 768)

In [ ]:
len(texts[0])

682

In [ ]:
patient_info_embed = model.encode(patients_info[0]['info'])
patient_text_embed.shape

# Format patient info 

## Create a dictonary of the medical records csvs and a list of patients 

In [5]:
csvs = {}
synthea =  "C:/Users/Jerem/OneDrive/Business/NN_health/Synthea/colab_records"
csv_loc = synthea+'/2022_04_04T07_13_58Z/'
csv_names = os.listdir(csv_loc)
for i, name in enumerate(csv_names):
    csv_names[i] = name[:-4]
    csvs[name[:-4]] = pd.read_csv(csv_loc+ name)


In [6]:

info_keys = ['allergies', 'medications', 'conditions', 'careplans', 
             'observations', 'procedures', 'immunizations', 'imaging_studies']

In [7]:
ids = np.array(csvs['procedures']['PATIENT'])

for info_key in info_keys:
    csv = csvs[info_key]
    if 'PATIENTID' in csv.keys():
        key = "PATIENTID"
    else:
        key = "PATIENT"
    uniq_ids = csv[key].unique()
    ids = np.concatenate((ids, np.array(uniq_ids)), axis=0)
ids = np.unique(ids)
print('Number of patients: ', ids.shape[0])

Number of patients:  1140


Next we will make a list filled with a dictonary for each patient (patients_info). Each dictonary will contain keys for the patient name, each record csv, and info. Info contains the full text of all the record csvs for this patient.

gt_patients_info is a list where each index corresponds to the same index in patients_info and contains the true insurance codes coressponding to each patinet.

gt_dict, Lastly gt_dict is a dictonary containing a key for each patient found in "ids" and all the insurance codes for the associated key/patients 



In [8]:
patients_info = []
gt_patients_info = []
gt_dict = {}
for patient in ids:
    patient_dict = {}
    patient_dict['name'] = patient
    info_list = []
    gt_list = []
    for info_key in info_keys:
        csv = csvs[info_key]
        # print(csv.keys())
        # print(info_key)
        if 'PATIENTID' in csv.keys():
            key = "PATIENTID"
        else:
            key = "PATIENT"
        try:
            data =np.array(csv.groupby(by = [key]).get_group(patient)['DESCRIPTION'])
            gt = np.array(csv.groupby(by = [key]).get_group(patient)['CODE'])
            patient_dict[info_key] = data
            info_list  = np.concatenate((info_list, data))
            gt_list = np.concatenate((gt_list, gt))
        except:
            
            pass
    patient_dict['info'] = info_list
    patients_info.append(patient_dict)
    gt_patients_info.append(gt_list) 
    gt_dict[patient] = gt_list

In [9]:
patients_info[0].keys()


dict_keys(['name', 'medications', 'conditions', 'careplans', 'observations', 'procedures', 'immunizations', 'info'])

In [13]:
synthea
text_loc
name

'text'

Text_dict simply contains all of the patients record names as keys and the file text content as the value

In [10]:
texts_dict = {}
texts = []
text_loc = synthea+'/text/text/'
text_names = os.listdir(text_loc)
for i, name in enumerate(text_names):
    text_names[i] = name[:-4]
    file = open(text_loc+ name, encoding="utf8")
    text=file.readlines()
    # with open(text_loc+ name) as f:
    #     text = f.readlines()
    texts_dict[name[:-4]] = text
    texts.append(text)

Map texts to embeded csv

In [14]:
length = 0
for text in texts:
    length += len(text)

In [35]:
os.listdir(os.path.abspath(text_loc+'../csv/'))

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Jerem\\OneDrive\\Business\\NN_health\\Synthea\\colab_records\\text\\csv'

# Text to csv

In [71]:
from datetime import datetime


def text_to_csv(text_loc, text_name, time=False):
    start = datetime.now()
    patient = text_name[:-4]
    csv_name = patient+'.csv'
    csv_loc = os.path.abspath(text_loc+'../encoded_text/'+csv_name)
    if os.path.exists(os.path.abspath(csv_loc)):   #not (csv_name in os.listdir(os.path.abspath(text_loc+'../encoded_text/'))):
        embeded_text= pd.read_csv(csv_loc)
        length = len(embeded_text)
    else:
        file = open(text_loc+ text_name, encoding="utf8")
        text=file.readlines()
        embeding = list(model.encode(text))
        length =len(text)
        column_patient = len(text)*[patient]
        embeded_text = pd.DataFrame({'Patient':column_patient, 'Text': text, 'Embeding': embeding}, index=np.arange(0,length, 1))
        embeded_text.to_csv(csv_loc)
    stop = datetime.now()
    if time:
        seconds = (stop-start)
        rate = seconds/length
        print("This file took: ", seconds)
        print('At a rate of ' + str(rate) +' per line')
    return embeded_text

In [73]:
c=[]
a= text_to_csv(text_loc, os.listdir(text_loc)[-3], time=True)
c.append(a)

This file took:  0:00:57.139560
At a rate of 0:00:00.058605 per line


In [75]:
len(embeded_dfs)

555

In [76]:
from tqdm.auto import tqdm
embeded_dfs =[]
text_loc = synthea+'/text/text/'
print(text_loc+'../encoded_text/')
for text_name in tqdm(os.listdir(text_loc)):
    embeded_dfs.append(text_to_csv(text_loc, text_name))

C:/Users/Jerem/OneDrive/Business/NN_health/Synthea/colab_records/text/text/../encoded_text/


  0%|          | 0/1140 [00:00<?, ?it/s]

In [23]:
embeded_texts = pd.DataFrame(columns=['patient','text','embeding'], index=np.arange(0,length, 1))

In [24]:
key = 'Leland44_Koelpin146_625911f7-0373-544d-0429-94111e133fcb'
text = texts_dict[key][0]
embeding=model.encode(text)

In [ ]:
text

In [ ]:
embeded_texts.iloc[0,:]= [key,text,embeding]

In [44]:
np.array(embeded_texts.iloc[2,:][0])

array(nan)

In [74]:
embeded_texts.iloc[i,:][0]

'Aaron697_McGlynn426_531c8d38-eddc-cd04-49e4-a3cf71dbee3e'

In [83]:
type(embeded_texts.iloc[946585,:][0]) == type('s')

False

In [82]:
np.isnan(embeded_texts.iloc[946585,:][0])

True

In [27]:
len(list(embeding))

314

In [30]:
embeded_texts.to_csv(text_loc+'../encoded_text/'+csv_name)

In [28]:
#embeded_texts = pd.DataFrame(columns=['patient','text','embeding'], index=np.arange(0,length, 1))
text_name = os.listdir(text_loc)[0]
patient = text_name[:-4]
csv_name = patient+'.csv'
if not (csv_name in os.listdir(text_loc+'../encoded_text/')):
    file = open(text_loc+ text_name, encoding="utf8")
    text=file.readlines()
    embeding = list(model.encode(text))
    length =len(text)
    column_patient = len(text)*[patient]
    embeded_text  = pd.DataFrame({'Patient':column_patient, 'Text': text, 'Embeding': embeding}, index=np.arange(0,length, 1))
    embeded_text.to_csv(text_loc+'../encoded_text/'+csv_name)

In [ ]:
def text_to_csv(text_loc, text_name):
    patient = text_name[:-4]
    csv_name = patient+'.csv'
    if not (csv_name in os.listdir(text_loc+'../csv/')):
        file = open(text_loc+ text_name, encoding="utf8")
        text=file.readlines()
        embeding = model.encode(text)
        length =len(text)
        column_patient = len(text)*[patient]
        embeded_text = pd.DataFrame({'Patient':column_patient, 'Text': text, 'Embeding': embeding}, index=np.arange(0,length, 1))
        embeded_text.to_csv(text_loc+'../encoded_text/'+csv_name)
    return embeded_text
    

In [ ]:
i = 0
for key in list(texts_dict.keys())[int(indexs[0]):int(indexs[1])]:
    patient = key
    for line in texts_dict[key]:
        try:
            if type(embeded_texts.iloc[i,:][0]) == type('s'):
                embeding = model.encode(line)
                embeded_texts.iloc[i,:]= [patient,line,embeding]
        except:
            print(embeded_texts.iloc[i,:][0])
    i += 1
    if (i%1000) == 0:
        print(i)


C:\Users\Jerem\anaconda3\envs\CogAI\lib\site-packages\pandas\core\internals\blocks.py:922: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


Adriana394_Mora209_ea0ef562-c9fc-3142-1bfc-2f30582c99b1
Adriana394_Mora209_ea0ef562-c9fc-3142-1bfc-2f30582c99b1
Adriana394_Mora209_ea0ef562-c9fc-3142-1bfc-2f30582c99b1


In [ ]:
embeded_texts

In [93]:
lenkey =len(list(texts_dict.keys()))
indexs = np.arange(0,lenkey, lenkey/4)

In [94]:
lenkey/4

285.0

In [ ]:
xprint(patients_info.append(patient_dict['info']))


None


In [86]:
len(list(texts_dict.keys()))
length

946590

In [ ]:
patients_info[0]['info']

In [66]:
len(texts[0])

682

In [25]:
texts_dict

TypeError: unhashable type: 'slice'